In [0]:
from google.colab import files
files.upload()

Saving employee_reviews.csv to employee_reviews (1).csv


In [0]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from keras import regularizers
import tensorflow as tf
from sklearn.model_selection import train_test_split
from collections import Counter
import numpy as np
import nltk
from nltk.corpus import stopwords
import keras
from sklearn.metrics import roc_curve
from sklearn.metrics import auc as aaa
import matplotlib.pyplot as plt
import pickle





def bake_pies(data):  
  
  cmap = plt.cm.GnBu

  current_size = data[data["status"] == "Current Employee"].count()
  former_size = data[data["status"] == "Former Employee"].count()
  ratio = current_size/ former_size

  for i in range(8,14):
    col = data.columns.tolist()[i]
    labels=data[col][data["status"] == "Former Employee"].value_counts().sort_index()
    current_data = data[col][data["status"] == "Current Employee"].value_counts().sort_index()
    former_data = data[col][data["status"] == "Former Employee"].value_counts().sort_index()
    colors = cmap(np.linspace(0., 1., len(labels)))
    
    
    plt.subplot(1, 2, 2)
    plt.pie(current_data, labels=labels.index, radius = ratio[0], startangle=90, colors = colors)
    
    plt.subplot(1, 2, 1)
    plt.pie(former_data, labels=labels.index, radius = 1, startangle=90, colors = colors)
    plt.title(col+"\n former vs current employees")
    
    plt.subplots_adjust(wspace=0.5)

   
    plt.show()

In [0]:
def remove_bias(data, feature, class1, class2):
  current_count = len(data[data[feature] == class1])
  former_count = len(data[data[feature] == class2])
  print(class1, current_count, class2, former_count)

  df_current = data[data[feature] == class1].sample(min(former_count, current_count))
  df_former = data[data[feature] == class2].sample(min(former_count, current_count))

  data = df_current.append(df_former)
  data = data.sample(frac=1)

  current_count = len(data[data[feature] == class1])
  former_count = len(data[data[feature] == class2])
  print(class1, current_count, class2, former_count)
  return data

In [0]:
def build_model(vocab_size, EMBEDDING_DIM):

  # Number of output nodes. This is binary classification, so 1 node.
  OUTPUT = 2

  model = keras.Sequential()

  model.add(keras.layers.Embedding(vocab_size, EMBEDDING_DIM))

  model.add(keras.layers.GlobalAveragePooling1D())

  model.add(keras.layers.Dense(16, activation=tf.nn.relu,
           kernel_regularizer=regularizers.l2(0.0001)))
  
  model.add(keras.layers.Dense(16, activation=tf.nn.relu))


  # model.add(keras.layers.Dense(NUM_NODES, activation=tf.nn.relu))

  model.add(keras.layers.Dense(OUTPUT*2, activation=tf.nn.sigmoid))
  model.add(keras.layers.Dense(OUTPUT, activation=tf.nn.sigmoid))

  model.summary()

  # Loss function and optimizer
  
  model.compile(optimizer=tf.train.AdamOptimizer(),
                loss='binary_crossentropy',
                metrics=['categorical_accuracy'])
  return model


## ACTUAL STUFF!

In [0]:
percent_occurred = 0.01
score_threshold = 3
len_threshold = 4
max_len = 50
min_word_length = 3

EMBEDDING_DIM = 8
remove_in_group_bias = False
use_nltk = False
combine = False

In [0]:
param_set ={}
param_set["use_nltk"] =[True, False]
param_set["remove_in_group_bias"] = [True, False]
param_set["combine"] = [True, False]
param_set["EMBEDDING_DIM"] = [4]
param_set["max_length"] = [50, 1000]

In [0]:
# param_set ={}
# param_set["use_nltk"] =[False]
# param_set["remove_in_group_bias"] = [ False]
# param_set["combine"] = [False]
# param_set["EMBEDDING_DIM"] = [4]
# param_set["max_length"] = [50]

We first read the raw csv data and remove the features that aren't useful for the decision making process, namely the **index** and the **links to the reviews**.

In [0]:
raw_data = pd.read_csv("employee_reviews.csv")
raw_data = raw_data.drop(raw_data.columns[0], axis = 1) # removing index
raw_data = raw_data.drop(raw_data.columns[-1], axis = 1) # removing links

def do_the_thing(raw_data, param_set):
  con = 1
#   histories = []
#   configs = []
#   models =[]
#   X_tests = []
#   y_tests = []
#   word_indices = []
  
  for combine in param_set["combine"]:
    for remove_in_group_bias in param_set["remove_in_group_bias"]:
      for use_nltk in param_set["use_nltk"]:
        for EMBEDDING_DIM in param_set["EMBEDDING_DIM"]:
          for max_len in param_set["max_length"]:
            
            data = raw_data.copy()
            
#             use_nltk = False
#             EMBEDDING_DIM=4
#             remove_in_group_bias = False
#             max_length = 50

            # convert numerical data
            for i in range(8,14):
              data[data.columns.tolist()[i]] = pd.to_numeric(data[data.columns.tolist()[i]], errors='ignore')

            # split job title and status
            status_split = data["job-title"].str.split(" - ").tolist()
            data['status'] = [x[0] for x in status_split]
            data['job-title'] = [x[1] for x in status_split]

            # resize class distribution for current /former employees
            if remove_in_group_bias:
              data = remove_bias(data, "status", "Current Employee", "Former Employee")

            # convert numerical data (NaN)
            for i in range(8,14):
              data[data.columns.tolist()[i]] = pd.to_numeric(data[data.columns.tolist()[i]], errors='coerce')

            # text data
            cols_text = ['company', 'summary', 'pros', 'cons', 'advice-to-mgmt']
            data_text = data[cols_text].copy()
            data_text['score'] = data['overall-ratings']

            # clean strings
            for key in cols_text:
              data_text[key] = data_text[key].str.lower()
              data_text[key] = data_text[key].fillna(" ")
              data_text[key] = data_text[key].str.replace('[^a-zA-Z\ ]+', ' ')

            # sentences
            all_sentences = []
            for key in cols_text[1:]:
              all_sentences = all_sentences + data_text[key].tolist()

            all_sentences_list = list(all_sentences)
            all_sentences = set(all_sentences_list)

            # words
            all_words = [word.lower() for sentence in all_sentences_list for word in sentence.split()]

            # remove words which are less than a certain length
            word_vocab = set(all_words)
            freq = [word for word in all_words if len(word) >= min_word_length] 
            counts = Counter(freq).most_common()
            word_freq = pd.DataFrame(counts, columns =['word', 'freq'])

            # remove words that appear less than a certain amont of times in the dataset
            threshold= len(raw_data)*(percent_occurred/100) # 0.05% times
            word_freq = word_freq[word_freq['freq'] > threshold]

            # remove stop words
            word_set_thresh = set(word_freq["word"])
            word_set_thresh.remove("microsoft")
            word_set_thresh.remove("google")
            word_set_thresh.remove("netflix")
            word_set_thresh.remove("facebook")
            word_set_thresh.remove("amazon")
            word_set_thresh.remove("apple")

            if use_nltk:
              nltk.download('stopwords')
              hola = set(stopwords.words('english'))
              word_set_thresh.difference_update(hola)
            else:
              word_set_thresh.difference_update({'the', 'and', 'are', 'that', 'for'})

            # tokenize the words colllection  
            for key in cols_text[1:]:
              data_text[key] = data_text[key].apply(lambda x: [word for word in x.split() if word in word_set_thresh])

            tokenize = keras.preprocessing.text.Tokenizer()
            vocab = data_text["cons"].tolist() + data_text["advice-to-mgmt"].tolist() + data_text["summary"].tolist() + data_text["pros"].tolist()
            tokenize.fit_on_texts(vocab)
            sentences = tokenize.texts_to_sequences(vocab)
            my_word_index = tokenize.word_index # encoder dict

            # convert words to token
            coded_inputs = []
            for key in cols_text[1:]:
              data_text[key] = data_text[key].apply(lambda x: [my_word_index[word] for word in x])
              coded_inputs = coded_inputs + data_text[key].tolist()

            # extra data by using combination of reviews (pros+cons+summary)
            if combine:
              z = []
              for index, row in data_text.iterrows():
                temp = np.concatenate([np.array(row.summary), np.array(row.pros)], axis=0)
                temp = np.concatenate([temp, row.cons], axis = 0)
              #   temp = np.concatenate([temp, row.advice-to-mgmt], axis = 0)  
                z.append(temp)
              data_text['combine'] = z


            # creating data set of pros, cons, summary (and pros+cons+summar)
            stacked_data = pd.DataFrame(columns=['summary', 'score'])
            stacked_data = stacked_data.append(data_text[["summary", "score"]])
            stacked_data = stacked_data.append(data_text[["pros", "score"]])
            stacked_data = stacked_data.append(data_text[["cons", "score"]])
            stacked_data = stacked_data.append(data_text[["advice-to-mgmt", "score"]])
            if combine:
              stacked_data = stacked_data.append(data_text[["combine", "score"]])
            stacked_data = stacked_data.reset_index(drop=True)

            # set lables to each text based on certain criteria
            data_len = len(data_text)
            stacked_data.iloc[:data_len, -2] = np.where(stacked_data.iloc[:data_len, -2] > score_threshold, 1, 0) # for all rows from 0 to datalen, assign score = 1 if rating > 2.5 # summary
            stacked_data.iloc[data_len:data_len*2, -2] = 1 # for all rows from datalen to datalen*2, assign score = 1 # pros
            stacked_data.iloc[data_len*2:data_len*3, -2] = 0 # for all rows from datalen*2 to datalen*3 assign score = 0 # cons
            stacked_data.iloc[data_len*3:data_len*4, -2] = np.where(stacked_data.iloc[data_len*3:data_len*4, -2] > score_threshold, 1, 0) # for adv to mgmt
            if combine:
              stacked_data.iloc[data_len*4:data_len*5, -2] = np.where(stacked_data.iloc[data_len*4:data_len*5, -2] > score_threshold, 1, 0) # for combine

            stacked_data["summary"] = stacked_data["summary"].fillna(stacked_data["pros"])
            stacked_data["summary"] = stacked_data["summary"].fillna(stacked_data["cons"])
            stacked_data["summary"] = stacked_data["summary"].fillna(stacked_data["advice-to-mgmt"])
            if combine:
              stacked_data["summary"] = stacked_data["summary"].fillna(stacked_data["combine"])
            stacked_data.isnull().sum()
            if combine:
              stacked_data = stacked_data.drop(columns=["pros", "cons", "advice-to-mgmt", "combine"])
            else:
              stacked_data = stacked_data.drop(columns=["pros", "cons", "advice-to-mgmt"])  

            # remove sentences less than a certain length based on frequency of lengths 
            stacked_data = stacked_data[stacked_data["summary"].str.len() > len_threshold]

            sentences_freq = stacked_data["summary"].str.len()
            sentences_freq.hist(bins = 10)

            # pad with 0s upto max_len
            padded_data = keras.preprocessing.sequence.pad_sequences(stacked_data["summary"],value=0, padding="post",maxlen=max_len)

            def create_dataset(df, data):
              df_data = pd.DataFrame(data)
              df_data['y'] = df['score'].values   
              return df_data

            data_set = create_dataset(stacked_data, padded_data)

            X_train, X_test, y_train, y_test = train_test_split(data_set.loc[:, data_set.columns != 'y'], data_set['y'], test_size=0.20, random_state=42,  shuffle=False)
            # one hot encoded binary targets
            y_train = pd.get_dummies(y_train)
            y_test = pd.get_dummies(y_test)

            # build model
            vocab_size = len(word_set_thresh)
            model = build_model(vocab_size, EMBEDDING_DIM)
            config = [combine, remove_in_group_bias, use_nltk, EMBEDDING_DIM, max_len]
            print("combine", config[0], "remove_in_group_bias", config[1] , 
            "use_nltk", config[2], "EMBEDDING_DIM", config[3], "max_len", config[4])
            print(config)
            history = model.fit(X_train,
                y_train,
                epochs=15,
                batch_size=128,
                validation_split=0.25,
                verbose=1)

            # pickle all realted data
            payload = [config, history, model, X_test, y_test, my_word_index]
            name = "config_v"+str(con)
            con = con + 1
            pickling_on = open(name,"wb")
            pickle.dump(payload, pickling_on)
            pickling_on.close()
            
            del payload


In [0]:
# configs, histories, models, X_tests, y_tests, my_word_index = do_the_thing(raw_data, param_set)
do_the_thing(raw_data, param_set)

In [0]:
from sklearn.metrics import classification_report

In [0]:
# pickle_off = open("config_32","rb")
# payload = pickle.load(pickle_off)

def auc_roc(model, y_pred, y_test, name):
  fpr, tpr, thresholds = roc_curve(np.argmax(y_test.values, axis =1), np.argmax(y_pred, axis = 1))
  auc_data = aaa(fpr, tpr)
  plt.plot(fpr, tpr, label='AUC={:.2f}'.format(auc_data))
  plt.xlabel('False positive rate')
  plt.ylabel('True positive rate')
  plt.title('ROC curve')
  plt.legend(loc='best')
  plt.savefig(name+"roc.jpg")
  plt.show()
  
# auc_roc
def metrics(y_pred, y_test):
  print(classification_report(np.argmax(y_test.values, axis =1),
                            np.argmax(y_pred, axis = 1),
                            target_names=["Negative", "Positive"]))

def learning_curves(history, name):
  acc = history.history['categorical_accuracy']
  val_acc = history.history['val_categorical_accuracy']
  loss = history.history['loss']
  val_loss = history.history['val_loss']
  print("Accuracy:", str(round(acc[-1], 2)), "Validation Accuracy:",str(round(val_acc[-1], 2)))
  acc_values = history.history['categorical_accuracy']
  val_acc_values = history.history['val_categorical_accuracy']

  loss_values = history.history['loss']
  loss_acc_values = history.history['val_loss']

  plt.plot( acc, 'b', label='Training acc')
  plt.plot(val_acc, 'r', label='Validation acc')

  plt.plot( loss, 'b--', label='Training loss')
  plt.plot(val_loss, 'r--', label='Validation loss')

  plt.title('Training and validation accuracy')
  plt.xlabel('Epochs')
  plt.ylabel('Accuracy')
  plt.legend()
  plt.savefig(name+"curve.jpg")
  plt.show()

def view(payload, name):
  config = payload[0]
  history= payload[1]
  model =payload[2]
  X_test = payload[3]
  y_test = payload[4]
  my_word_index = payload[5]
  
  
  y_pred = model.predict(X_test)
  print("###########################################################################")
  print(" combine:", config[0], "\n remove_in_group_bias:", config[1] , "\n use_nltk:", config[2], "\n EMBEDDING_DIM:", config[3], "\n max_len:", config[4])
  auc_roc(model, y_pred, y_test, name)
  metrics(y_pred, y_test)
  learning_curves(history, name)

In [0]:
import fnmatch
import os

for file in os.listdir('/content/'):
    if fnmatch.fnmatch(file, 'config_v*'):
        print(file)
        pickle_off = open(file,"rb")
        payload = pickle.load(pickle_off)
        view(payload, file)

In [0]:
# view(payload)

In [0]:
for y_,y, s in zip(model.predict(X_sample), y_sample.values, range(len(y_sample))):
  print(y_, np.argmax(y_), np.argmax(y), decode(X_train.iloc[s].values))

[0.15362889 0.84866023] 1 1 moving speed light burn out inevitable 
[0.87449324 0.12546366] 0 0 great balance between big company security fun fast moving projects 
[0.42996043 0.5708599 ] 1 0 best place worked also most demanding 
[0.9106748  0.08961487] 0 0 unique one kind dream job 
[0.81201446 0.1875422 ] 0 0 great company with down sides 
[0.1706591  0.83126956] 1 1 great but great can always better 
[0.01344371 0.9865833 ] 1 1 not perfect but still best place world work 
[0.59770274 0.40236256] 0 0 great perks bad work life balance 
[0.05477118 0.94636154] 1 1 make sure you know what role you applying 
[0.84842527 0.15126932] 0 1 good company can long hours 
[0.14677167 0.85508776] 1 1 great benefits but large enough get lost 
[0.02415434 0.9762231 ] 1 1 good work life balance perks 
[0.63689065 0.3626784 ] 0 1 amazing place work challenging environment 
[0.5051973  0.49542665] 0 0 great perks interesting colleagues zero work life balance 
[0.03299329 0.9676132 ] 1 1 perks work e

In [0]:
# confusion matrix
# model weakness
# threshold decisions
# metrics vs thresholds
# sarcasm identification!